## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Currently,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tiksi,71.6872,128.8694,Clouds,38.03,79,100,9.89,RU,2021-07-17 19:10:29
1,1,Sungaipenuh,-2.0833,101.3833,Clouds,63.99,97,50,1.83,ID,2021-07-17 19:10:29
2,2,Rikitea,-23.1203,-134.9692,Rain,72.61,78,98,26.89,PF,2021-07-17 19:10:29
3,3,Mataura,-46.1927,168.8643,Clouds,44.47,99,96,2.19,NZ,2021-07-17 19:10:29
4,4,Gambela,8.2500,34.5833,Rain,69.66,98,100,3.98,ET,2021-07-17 19:10:30


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Currently,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Rikitea,-23.1203,-134.9692,Rain,72.61,78,98,26.89,PF,2021-07-17 19:10:29
4,4,Gambela,8.2500,34.5833,Rain,69.66,98,100,3.98,ET,2021-07-17 19:10:30
5,5,Morrope,-6.5417,-80.0111,Clouds,77.11,58,58,14.92,PE,2021-07-17 19:10:30
6,6,Wladyslawowo,54.7909,18.4009,Clear,71.15,85,2,8.08,PL,2021-07-17 19:10:30
7,7,Kodiak,57.7900,-152.4072,Clear,65.71,63,1,3.44,US,2021-07-17 19:10:30
9,9,Kapaa,22.0752,-159.3190,Rain,82.38,80,90,1.99,US,2021-07-17 19:08:21
11,11,Neka,36.6508,53.2990,Clouds,77.45,81,59,3.29,IR,2021-07-17 19:10:31
12,12,Bambous Virieux,-20.3428,57.7575,Rain,71.22,74,42,17.00,MU,2021-07-17 19:10:31
14,14,Kitgum,3.2783,32.8867,Rain,68.86,79,99,2.66,UG,2021-07-17 19:09:17
16,16,Jamestown,42.0970,-79.2353,Clouds,78.12,94,90,13.80,US,2021-07-17 19:10:32


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       366
City          366
Lat           366
Lng           366
Currently     366
Max Temp      366
Humidity      366
Cloudiness    366
Wind Speed    366
Country       362
Date          366
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Currently", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Currently,Lat,Lng,Hotel Name
2,Rikitea,PF,72.61,Rain,-23.1203,-134.9692,
4,Gambela,ET,69.66,Rain,8.2500,34.5833,
5,Morrope,PE,77.11,Clouds,-6.5417,-80.0111,
6,Wladyslawowo,PL,71.15,Clear,54.7909,18.4009,
7,Kodiak,US,65.71,Clear,57.7900,-152.4072,
9,Kapaa,US,82.38,Rain,22.0752,-159.3190,
11,Neka,IR,77.45,Clouds,36.6508,53.2990,
12,Bambous Virieux,MU,71.22,Rain,-20.3428,57.7575,
14,Kitgum,UG,68.86,Rain,3.2783,32.8867,
16,Jamestown,US,78.12,Clouds,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
nancount = 0
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city (populate with nan).
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        nancount += 1

print("No hotel info returned for " + str(nancount) + " locations.")

No hotel info returned for 30 locations.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Currently</dt><dd>{Currently}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'gmaps' is not defined